### Check GPU availability

In [1]:
!nvidia-smi

Mon Feb  4 18:57:38 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.145                Driver Version: 384.145                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   26C    P8    17W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 23%   32C    P8    17W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

### Import libraries

In [2]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import copy
import numpy as np
import time
import json
import plotly
import logging
logging.getLogger().setLevel(logging.INFO)

from pprint import pprint
from tqdm import tqdm_notebook
from idst_util import trivial
from idst_util import dstc2

from plotly.graph_objs import Scatter, Layout
from plotly.graph_objs.layout import Margin
plotly.offline.init_notebook_mode(connected = True)

[nltk_data] Downloading package punkt to /home/is/andrei-
[nltk_data]     cc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Check DSTC2 availability

In [3]:
trivial.print_idst()
dstc2.check()

INFO:root:+--------------------------------+
INFO:root:|         _ ____  ___________    |
INFO:root:|        (_) __ \/ ___/_  __/    |
INFO:root:|       / / / / /\__ \ / /       |
INFO:root:|      / / /_/ /___/ // /        |
INFO:root:|     /_/_____//____//_/         |
INFO:root:|                                |
INFO:root:+--------------------------------+
INFO:root:|Incremental Dialog State Tracker|
INFO:root:+--------------------------------+
INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|         Data Checker           |
INFO:root:+--------------------------------+
INFO:root:Looking for dstc2 directory in .
INFO:root:dstc2 was found!
INFO:root:Looking for dstc2_traindev directory in ./dstc2
INFO:root:dstc2_traindev was found!
INFO:root:Looking for dstc2_test directory in ./dstc2
INFO:root:dstc2_test was found!
INFO:root:Looking for dstc2_scripts directory in ./dstc2
INFO:root:dstc2_scripts was found!
INFO:root:Done!


### Retrieve data

In [4]:
raw_X_train, raw_Y_train, \
raw_X_dev, raw_Y_dev, \
raw_X_test, raw_Y_test, \
ontology = dstc2.retrieve_raw_datasets(train_data_augmentation = True)

INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|       Dataset Retrieval        |
INFO:root:+--------------------------------+
INFO:root:Reading dstc2_train.flist, dstc2_dev.flist and ontology_dstc2.json
INFO:root:Asserted 1612 dialogs for dstc2_train.flist
INFO:root:Asserted 506 dialogs for dstc2_dev.flist
INFO:root:Extracting raw train features


INFO:root:Extracting raw dev features


INFO:root:Reading dstc2_test.flist
INFO:root:Asserted 1117 dialogs for dstc2_test.flist
INFO:root:Extracting raw test features


INFO:root:Done!


### Set device

In [ ]:
logging.info("+--------------------------------+")
logging.info("|            Baseline            |")
logging.info("+--------------------------------+")

GPU_ID = 1
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
#DEVICE = "cpu"
if DEVICE == "cpu":
    logging.warning("Running on CPU")
else:
    logging.info("Running on GPU {}".format(GPU_ID))

### Models utilities

In [ ]:
def get_index_and_score(turn, token_to_index, mode, device):
    indices = []
    scores = []
    if mode == "train":
        for system_token, system_token_score in turn["system"]:
            indices.append(token_to_index[system_token])
            scores.append(system_token_score)
        for user_token, user_token_score in turn["user"]:
            if np.random.binomial(n = 1, p = 0.1) == 1:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[user_token])
            scores.append(user_token_score)
    else:
        tokens_scores = turn["system"] + turn["user"]
        for token, score in tokens_scores:
            if token not in token_to_index:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[token])
            scores.append(score)
    assert len(indices) == len(scores)
    return torch.tensor(indices, dtype = torch.long, device = device), torch.tensor(scores, dtype = torch.float, device = device)

class EarlyStopping():
    
    def __init__(self, min_delta = 0, patience = 0):
        
        self.min_delta = min_delta
        self.patience = patience
        self.wait = 0
        self.stopped_epoch = 0
        self.best = -np.Inf
        self.stop_training = False
    
    def on_epoch_end(self, epoch, current_value):
        if np.greater((current_value - self.min_delta), self.best):
            self.best = current_value
            self.wait = 0
        else:
            self.wait += 1
            if self.wait > self.patience:
                self.stopped_epoch = epoch
                self.stop_training = True
        return self.stop_training

def make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset, percentage = 1.0):
    
    model_GoalPricerange = model_GoalPricerange.eval()
    model_GoalArea = model_GoalArea.eval()
    model_GoalName = model_GoalName.eval()
    model_GoalFood = model_GoalFood.eval()
    model_Requested = model_Requested.eval()
    model_Method = model_Method.eval()
    
    with torch.no_grad():
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        for raw_X_dialog, raw_Y_dialog in tqdm_notebook(zip(raw_X, raw_Y), total = len(raw_X)):
            
            model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
            model_GoalArea.hidden = model_GoalArea.init_hidden()
            model_GoalName.hidden = model_GoalName.init_hidden()
            model_GoalFood.hidden = model_GoalFood.init_hidden()
            model_Requested.hidden = model_Requested.init_hidden()
            model_Method.hidden = model_Method.init_hidden()
            
            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for raw_X_turn, raw_Y_turn in zip(raw_X_dialog["turns"], raw_Y_dialog["turns"]):
                turn = {}
                turn["goal-labels"] = {}

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)
                
                incremental_index = int(np.around(percentage * len(indices))) - 1
                
                goal_foods = model_GoalFood(indices, scores)
                goal_food = goal_foods[incremental_index]
                
                goal_priceranges = model_GoalPricerange(indices, scores)
                goal_pricerange = goal_priceranges[incremental_index]
                
                goal_names = model_GoalName(indices, scores)
                goal_name = goal_names[incremental_index]
                
                goal_areas = model_GoalArea(indices, scores)
                goal_area = goal_areas[incremental_index]
                
                requesteds = model_Requested(indices, scores)
                requested = requesteds[incremental_index]
                
                methods = model_Method(indices, scores)
                method = methods[incremental_index]
                
                turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food, ontology)
                turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange, ontology)
                turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name, ontology)
                turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area, ontology)
                turn["requested-slots"] = retrieve_output_Requested(requested, ontology)
                turn["method-label"] = retrieve_output_Method(method, ontology)
                
                session["turns"].append(turn)
                
            tracker_json["sessions"].append(session)
            
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        
        return tracker_json

def get_scores(tracker, dataset, predictor = False):
    goal_pricerange_accuracy = None
    goal_pricerange_l2 = None
    goal_area_accuracy = None
    goal_area_l2 = None
    goal_name_accuracy = None
    goal_name_l2 = None
    goal_food_accuracy = None
    goal_food_l2 = None
    
    goal_accuracy = None
    goal_l2 = None
    requested_accuracy = None
    requested_l2 = None
    method_accuracy = None
    method_l2 = None
    
    with open("tracker_panfmr_incremental.json", "w") as tracker_file:
        json.dump(tracker, tracker_file)
    
    if predictor:
        
        if dataset == "dstc2_train":
            !python2 dstc2/dstc2_scripts/score_predictor.py\
            --dataset dstc2_train\
            --dataroot dstc2/dstc2_traindev/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
        elif dataset == "dstc2_dev":
            !python2 dstc2/dstc2_scripts/score_predictor.py\
            --dataset dstc2_dev\
            --dataroot dstc2/dstc2_traindev/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
        else:
            !python2 dstc2/dstc2_scripts/score_predictor.py\
            --dataset dstc2_test\
            --dataroot dstc2/dstc2_test/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
            
    else: # predictor == False
        
        if dataset == "dstc2_train":
            !python2 dstc2/dstc2_scripts/score.py\
            --dataset dstc2_train\
            --dataroot dstc2/dstc2_traindev/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
        elif dataset == "dstc2_dev":
            !python2 dstc2/dstc2_scripts/score.py\
            --dataset dstc2_dev\
            --dataroot dstc2/dstc2_traindev/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
        else:
            !python2 dstc2/dstc2_scripts/score.py\
            --dataset dstc2_test\
            --dataroot dstc2/dstc2_test/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
        
    score_file_cat = !cat tracker_panfmr_incremental.score.csv
    
    for line in score_file_cat:
        if line.startswith("goal.pricerange, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_pricerange_accuracy = float(value)
        if line.startswith("goal.pricerange, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_pricerange_l2 = float(value)
        if line.startswith("goal.area, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_area_accuracy = float(value)
        if line.startswith("goal.area, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_area_l2 = float(value)
        if line.startswith("goal.name, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_name_accuracy = float(value)
        if line.startswith("goal.name, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_name_l2 = float(value)
        if line.startswith("goal.food, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_food_accuracy = float(value)
        if line.startswith("goal.food, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_food_l2 = float(value)
        if line.startswith("goal.joint, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_accuracy = float(value)
        if line.startswith("goal.joint, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_l2 = float(value)
        if line.startswith("requested.all, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                requested_accuracy = float(value)
        if line.startswith("requested.all, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                requested_l2 = float(value)
        if line.startswith("method, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                method_accuracy = float(value)
        if line.startswith("method, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                method_l2 = float(value)
    
    scores_dict = {}
    scores_dict["goal_pricerange_accuracy"] = goal_pricerange_accuracy
    scores_dict["goal_pricerange_l2"] = goal_pricerange_l2
    scores_dict["goal_area_accuracy"] = goal_area_accuracy
    scores_dict["goal_area_l2"] = goal_area_l2
    scores_dict["goal_name_accuracy"] = goal_name_accuracy
    scores_dict["goal_name_l2"] = goal_name_l2
    scores_dict["goal_food_accuracy"] = goal_food_accuracy
    scores_dict["goal_food_l2"] = goal_food_l2
    scores_dict["goal_accuracy"] = goal_accuracy
    scores_dict["goal_l2"] = goal_l2
    scores_dict["requested_accuracy"] = requested_accuracy
    scores_dict["requested_l2"] = requested_l2
    scores_dict["method_accuracy"] = method_accuracy
    scores_dict["method_l2"] = method_l2
    
    return scores_dict

def retrieve_gold_GoalFood(raw_Y, ontology, device):
    ontology_informable_food = ontology["informable"]["food"]
    raw_goal_food = raw_Y["goal"]["food"]
    goal_food = 0
    if raw_goal_food != None:
        if raw_goal_food == "dontcare":
            goal_food = 1
        else:    
            goal_food = ontology_informable_food.index(raw_goal_food) + 2
    return torch.tensor([goal_food], dtype = torch.long, device = device)

def retrieve_output_GoalFood(output_tensor, ontology):
    ontology_informable_food = ontology["informable"]["food"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_food_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_food_dict["dontcare"] = output_tensor[1].item() 
    for index in range(len(output_tensor) - 2):
        goal_food_dict[ontology_informable_food[index]] = output_tensor[index + 2].item()
    return goal_food_dict

def retrieve_gold_GoalPricerange(raw_Y, ontology, device):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    raw_goal_pricerange = raw_Y["goal"]["pricerange"]
    goal_pricerange = 0
    if raw_goal_pricerange != None:
        if raw_goal_pricerange == "dontcare":
            goal_pricerange = 1
        else:    
            goal_pricerange = ontology_informable_pricerange.index(raw_goal_pricerange) + 2
    return torch.tensor([goal_pricerange], dtype = torch.long, device = device)

def retrieve_output_GoalPricerange(output_tensor, ontology):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_pricerange_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_pricerange_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):     
        goal_pricerange_dict[ontology_informable_pricerange[index]] = output_tensor[index + 2].item()
    return goal_pricerange_dict

def retrieve_gold_GoalName(raw_Y, ontology, device):
    ontology_informable_name = ontology["informable"]["name"]
    raw_goal_name = raw_Y["goal"]["name"]
    goal_name = 0
    if raw_goal_name != None:
        if raw_goal_name == "dontcare":
            goal_name = 1
        else:    
            goal_name = ontology_informable_name.index(raw_goal_name) + 2
    return torch.tensor([goal_name], dtype = torch.long, device = device)

def retrieve_output_GoalName(output_tensor, ontology):
    ontology_informable_name = ontology["informable"]["name"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_name_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_name_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_name_dict[ontology_informable_name[index]] = output_tensor[index + 2].item()
        
    return goal_name_dict

def retrieve_gold_GoalArea(raw_Y, ontology, device):
    ontology_informable_area = ontology["informable"]["area"]
    raw_goal_area = raw_Y["goal"]["area"]
    goal_area = 0
    if raw_goal_area != None:
        if raw_goal_area == "dontcare":
            goal_area = 1
        else:    
            goal_area = ontology_informable_area.index(raw_goal_area) + 2
    return torch.tensor([goal_area], dtype = torch.long, device = device)

def retrieve_output_GoalArea(output_tensor, ontology):
    ontology_informable_area = ontology["informable"]["area"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_area_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_area_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_area_dict[ontology_informable_area[index]] = output_tensor[index + 2].item()
        
    return goal_area_dict

def retrieve_gold_Method(raw_Y, ontology, device):
    ontology_method = ontology["method"]
    raw_gold_method = raw_Y["method"]
    gold_method = ontology_method.index(raw_gold_method)
    return torch.tensor([gold_method], dtype = torch.long, device = device)

def retrieve_output_Method(output_tensor, ontology):
    ontology_method = ontology["method"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    method_dict = {}
    
    for index in range(len(output_tensor)):
        method_dict[ontology_method[index]] = output_tensor[index].item()
    
    return method_dict

def retrieve_gold_Requested(raw_Y, ontology, device):
    ontology_requestable = ontology["requestable"]
    raw_gold_requested = raw_Y["requested"]
    gold_requested = np.zeros(len(ontology_requestable), dtype = float)
    if len(raw_gold_requested) != 0:
        for requested in raw_gold_requested:
            gold_requested[ontology_requestable.index(requested)] = 1.0
    return torch.tensor([gold_requested], dtype = torch.float, device = device)

def retrieve_output_Requested(output_tensor, ontology):
    ontology_requestable = ontology["requestable"]
    output_tensor = output_tensor.view(-1)
    requested_dict = {}
    for index in range(len(output_tensor)):
        probability_value = output_tensor[index].item()
        if np.greater_equal(probability_value, 0.5):
            requested_dict[ontology_requestable[index]] = probability_value
    return requested_dict

### Create vocabularies

In [ ]:
logging.info("+--------------------------------+")
logging.info("|          Vocabulary            |")
logging.info("+--------------------------------+")
logging.info("Creating token_to_index, index_to_token and token_to_count dictionaries")

token_to_index = {"<unk>": 0}
index_to_token = {0: "<unk>"}
token_to_count = {"<unk>": 1}

for raw_train_dialog in tqdm_notebook(raw_X_train):
    
    for raw_train_turn in raw_train_dialog["turns"]:
        
        tokens_scores = raw_train_turn["system"] + raw_train_turn["user"]
        
        for token_score in tokens_scores:
            token = token_score[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
                
assert len(token_to_index) == len(index_to_token)
assert len(token_to_index) == len(token_to_count)

### Execution configuration

In [ ]:
logging.info("+--------------------------------+")
logging.info("|         Configuration          |")
logging.info("+--------------------------------+")

VOCABULARY_SIZE = len(token_to_index)
logging.info("VOCABULARY_SIZE:\t\t\t{}".format(VOCABULARY_SIZE))

# NOTE: +2 because of null and dontcare
GOAL_FOOD_DIM = len(ontology["informable"]["food"]) + 2 
GOAL_PRICERANGE_DIM = len(ontology["informable"]["pricerange"]) + 2
GOAL_NAME_DIM = len(ontology["informable"]["name"]) + 2
GOAL_AREA_DIM = len(ontology["informable"]["area"]) + 2
METHOD_DIM = len(ontology["method"])
REQUESTED_DIM = len(ontology["requestable"])
EMBEDDING_DIM = 170
ALTERED_EMBEDDING_DIM = 300
HIDDEN_DIM = 100
NUM_EPOCHS = 50
BATCH_SIZE = 10
PREDICTOR_DIM = 2
PREDICTOR_LOSS_FUNCTION = nn.CrossEntropyLoss()
GOAL_LOSS_FUNCTION = nn.CrossEntropyLoss()
METHOD_LOSS_FUNCTION = nn.CrossEntropyLoss()
REQUESTED_LOSS_FUNCTION = nn.BCELoss()
logging.info("GOAL_FOOD_DIM:\t\t\t{}".format(GOAL_FOOD_DIM))
logging.info("GOAL_PRICERANGE_DIM:\t\t\t{}".format(GOAL_PRICERANGE_DIM))
logging.info("GOAL_NAME_DIM:\t\t\t{}".format(GOAL_NAME_DIM))
logging.info("GOAL_AREA_DIM:\t\t\t{}".format(GOAL_AREA_DIM))
logging.info("METHOD_DIM:\t\t\t\t{}".format(METHOD_DIM))
logging.info("REQUESTED_DIM:\t\t\t{}".format(REQUESTED_DIM))
logging.info("EMBEDDING_DIM:\t\t\t{}".format(EMBEDDING_DIM))
logging.info("ALTERED_EMBEDDING_DIM:\t\t{}".format(ALTERED_EMBEDDING_DIM))
logging.info("HIDDEN_DIM:\t\t\t\t{}".format(HIDDEN_DIM))
logging.info("NUM_EPOCHS:\t\t\t\t{}".format(NUM_EPOCHS))
logging.info("BATCH_SIZE:\t\t\t\t{}".format(BATCH_SIZE))
logging.info("PREDICTOR_DIM:\t\t\t{}".format(PREDICTOR_DIM))
logging.info("PREDICTOR_LOSS_FUNCTION:\t\t{}".format(PREDICTOR_LOSS_FUNCTION))
logging.info("GOAL_LOSS_FUNCTION:\t\t\t{}".format(GOAL_LOSS_FUNCTION))
logging.info("METHOD_LOSS_FUNCTION:\t\t\t{}".format(METHOD_LOSS_FUNCTION))
logging.info("REQUESTED_LOSS_FUNCTION:\t\t{}".format(REQUESTED_LOSS_FUNCTION))

### Goal Pricerange Model

In [ ]:
class GoalPricerangeModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_pricerange_dim,
                 device):
        
        super(GoalPricerangeModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_pricerange_dim = goal_pricerange_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_pricerange_classifier = nn.Linear(in_features = hidden_dim,
                                                    out_features = goal_pricerange_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_pricerange = F.log_softmax(self.goal_pricerange_classifier(lstm_out).view(-1, self.goal_pricerange_dim), dim = 1) 
        
        return goal_pricerange

model_GoalPricerange = GoalPricerangeModel(vocabulary_size = VOCABULARY_SIZE,
                                           embedding_dim = EMBEDDING_DIM,
                                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                           hidden_dim = HIDDEN_DIM,
                                           goal_pricerange_dim = GOAL_PRICERANGE_DIM,
                                           device = DEVICE)

model_GoalPricerange = model_GoalPricerange.to(DEVICE)

optimizer_GoalPricerangeModel = optim.Adam(model_GoalPricerange.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Area Model

In [ ]:
class GoalAreaModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_area_dim,
                 device):
        
        super(GoalAreaModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_area_dim = goal_area_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_area_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_area_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_area = F.log_softmax(self.goal_area_classifier(lstm_out).view(-1, self.goal_area_dim), dim = 1)
        
        return goal_area

model_GoalArea = GoalAreaModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_area_dim = GOAL_AREA_DIM,
                               device = DEVICE)

model_GoalArea = model_GoalArea.to(DEVICE)

optimizer_GoalAreaModel = optim.Adam(model_GoalArea.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Name Model

In [ ]:
class GoalNameModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_name_dim,
                 device):
        
        super(GoalNameModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_name_dim = goal_name_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_name_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_name_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_name = F.log_softmax(self.goal_name_classifier(lstm_out).view(-1, self.goal_name_dim), dim = 1)
        
        return goal_name

model_GoalName = GoalNameModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_name_dim = GOAL_NAME_DIM,
                               device = DEVICE)

model_GoalName = model_GoalName.to(DEVICE)

optimizer_GoalNameModel = optim.Adam(model_GoalName.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Food Model

In [ ]:
class GoalFoodModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_food_dim,
                 device):
        
        super(GoalFoodModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_food_dim = goal_food_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_food_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_food_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_food = F.log_softmax(self.goal_food_classifier(lstm_out).view(-1, self.goal_food_dim), dim = 1)        
        
        return goal_food

model_GoalFood = GoalFoodModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_food_dim = GOAL_FOOD_DIM,
                               device = DEVICE)

model_GoalFood = model_GoalFood.to(DEVICE)

optimizer_GoalFoodModel = optim.Adam(model_GoalFood.parameters(), lr = 1e-3, amsgrad = True) 

### Requested Model

In [ ]:
class RequestedModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 requested_dim,
                 device):
        
        super(RequestedModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.requested_dim = requested_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.requested_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = requested_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        requested = torch.sigmoid(self.requested_classifier(lstm_out).view(-1, self.requested_dim))
        
        return requested

model_Requested = RequestedModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 requested_dim = REQUESTED_DIM,
                                 device = DEVICE)

model_Requested = model_Requested.to(DEVICE)

optimizer_RequestedModel = optim.Adam(model_Requested.parameters(), lr = 1e-3, amsgrad = True) 

### Method Model

In [ ]:
class MethodModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 method_dim,
                 device):
        
        super(MethodModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.method_dim = method_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.method_classifier = nn.Linear(in_features = hidden_dim,
                                           out_features = method_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        method = F.log_softmax(self.method_classifier(lstm_out).view(-1, self.method_dim), dim = 1)
        
        return method

model_Method = MethodModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           method_dim = METHOD_DIM,
                           device = DEVICE)

model_Method = model_Method.to(DEVICE)

optimizer_MethodModel = optim.Adam(model_Method.parameters(), lr = 1e-3, amsgrad = True) 

### Train Goal Pricerange Model

In [ ]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

goal_pricerange_early_stopping = EarlyStopping(patience = 4)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalPricerange = model_GoalPricerange.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        optimizer_GoalPricerangeModel.zero_grad()

        dialogs_loss = 0
    
        for raw_X_train_dialog, raw_Y_train_dialog in zip(raw_X_train[train_indices], raw_Y_train[train_indices]):

            model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()

            for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                goal_priceranges = model_GoalPricerange(indices, scores)

                for goal_pricerange in goal_priceranges:

                    goal_pricerange = goal_pricerange.unsqueeze(0)

                    loss_goal_pricerange = GOAL_LOSS_FUNCTION(goal_pricerange,
                                                              retrieve_gold_GoalPricerange(raw_Y_train_turn,
                                                                                           ontology = ontology,
                                                                                           device = DEVICE))
                    dialogs_loss += loss_goal_pricerange

        dialogs_loss.backward(retain_graph = False)

        optimizer_GoalPricerangeModel.step()
        
    dev_tracker = make_tracker(model_GoalPricerange,
                       model_GoalArea,
                       model_GoalName,
                       model_GoalFood,
                       model_Requested,
                       model_Method,
                       raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
    dev_scores_dict = get_scores(dev_tracker, dataset = "dstc2_dev")
    
    logging.info("DEV Goal Pricerange Acc: {}".format(dev_scores_dict["goal_pricerange_accuracy"]))
    
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_accuracy"],
                                                                 np.around(dev_scores_dict["goal_accuracy"], decimals = 2),
                                                                 dev_scores_dict["requested_accuracy"],
                                                                 np.around(dev_scores_dict["requested_accuracy"], decimals = 2),
                                                                 dev_scores_dict["method_accuracy"],
                                                                 np.around(dev_scores_dict["method_accuracy"], decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_l2"],
                                                                np.around(dev_scores_dict["goal_l2"], decimals = 2),
                                                                dev_scores_dict["requested_l2"],
                                                                np.around(dev_scores_dict["requested_l2"], decimals = 2),
                                                                dev_scores_dict["method_l2"],
                                                                np.around(dev_scores_dict["method_l2"], decimals = 2)))

    goal_pricerange_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_pricerange_accuracy"]))
    
    if goal_pricerange_early_stopping.wait == 0:
        torch.save(model_GoalPricerange.state_dict(), "model_PANFMR_incremental_GoalPricerange.pt")
    
    if goal_pricerange_early_stopping.stop_training:
        break

### Load Goal Pricerange Model

In [ ]:
model_GoalPricerange = GoalPricerangeModel(vocabulary_size = VOCABULARY_SIZE,
                                           embedding_dim = EMBEDDING_DIM,
                                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                           hidden_dim = HIDDEN_DIM,
                                           goal_pricerange_dim = GOAL_PRICERANGE_DIM,
                                           device = DEVICE)

model_GoalPricerange = model_GoalPricerange.to(DEVICE)
model_GoalPricerange.load_state_dict(torch.load("model_PANFMR_incremental_GoalPricerange.pt"))
model_GoalPricerange.eval()

### Print scores

In [ ]:
dev_tracker = make_tracker(model_GoalPricerange,
                           model_GoalArea,
                           model_GoalName,
                           model_GoalFood,
                           model_Requested,
                           model_Method,
                           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
get_scores(dev_tracker, dataset = "dstc2_dev")

In [ ]:
test_tracker = make_tracker(model_GoalPricerange,
                            model_GoalArea,
                            model_GoalName,
                            model_GoalFood,
                            model_Requested,
                            model_Method,
                            raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = 1.0)
get_scores(test_tracker, dataset = "dstc2_test")

### Train Goal Area Model

In [ ]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

goal_area_early_stopping = EarlyStopping(patience = 4)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalArea = model_GoalArea.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        optimizer_GoalAreaModel.zero_grad()

        dialogs_loss = 0
    
        for raw_X_train_dialog, raw_Y_train_dialog in zip(raw_X_train[train_indices], raw_Y_train[train_indices]):

            model_GoalArea.hidden = model_GoalArea.init_hidden()

            for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                goal_areas = model_GoalArea(indices, scores)

                for goal_area in goal_areas:

                    goal_area = goal_area.unsqueeze(0)

                    loss_goal_area = GOAL_LOSS_FUNCTION(goal_area,
                                                        retrieve_gold_GoalArea(raw_Y_train_turn,
                                                                               ontology = ontology,
                                                                               device = DEVICE))
                    dialogs_loss += loss_goal_area

        dialogs_loss.backward(retain_graph = False)

        optimizer_GoalAreaModel.step()
    
    dev_tracker = make_tracker(model_GoalPricerange,
                   model_GoalArea,
                   model_GoalName,
                   model_GoalFood,
                   model_Requested,
                   model_Method,
                   raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
    dev_scores_dict = get_scores(dev_tracker, dataset = "dstc2_dev")

    logging.info("DEV Goal Area Acc: {}".format(dev_scores_dict["goal_area_accuracy"]))

    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_accuracy"],
                                                                 np.around(dev_scores_dict["goal_accuracy"], decimals = 2),
                                                                 dev_scores_dict["requested_accuracy"],
                                                                 np.around(dev_scores_dict["requested_accuracy"], decimals = 2),
                                                                 dev_scores_dict["method_accuracy"],
                                                                 np.around(dev_scores_dict["method_accuracy"], decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_l2"],
                                                                np.around(dev_scores_dict["goal_l2"], decimals = 2),
                                                                dev_scores_dict["requested_l2"],
                                                                np.around(dev_scores_dict["requested_l2"], decimals = 2),
                                                                dev_scores_dict["method_l2"],
                                                                np.around(dev_scores_dict["method_l2"], decimals = 2)))

    goal_area_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_area_accuracy"]))
    
    if goal_area_early_stopping.wait == 0:
        torch.save(model_GoalArea.state_dict(), "model_PANFMR_incremental_GoalArea.pt")
    
    if goal_area_early_stopping.stop_training:
        break

### Load Goal Area Model

In [ ]:
model_GoalArea = GoalAreaModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_area_dim = GOAL_AREA_DIM,
                               device = DEVICE)

model_GoalArea = model_GoalArea.to(DEVICE)
model_GoalArea.load_state_dict(torch.load("model_PANFMR_incremental_GoalArea.pt"))
model_GoalArea.eval()

### Print scores

In [ ]:
dev_tracker = make_tracker(model_GoalPricerange,
                           model_GoalArea,
                           model_GoalName,
                           model_GoalFood,
                           model_Requested,
                           model_Method,
                           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
get_scores(dev_tracker, dataset = "dstc2_dev")

In [ ]:
test_tracker = make_tracker(model_GoalPricerange,
                            model_GoalArea,
                            model_GoalName,
                            model_GoalFood,
                            model_Requested,
                            model_Method,
                            raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = 1.0)
get_scores(test_tracker, dataset = "dstc2_test")

### Train Goal Name Model

In [ ]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

goal_name_early_stopping = EarlyStopping(patience = 4)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalName = model_GoalName.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        optimizer_GoalNameModel.zero_grad()

        dialogs_loss = 0
    
        for raw_X_train_dialog, raw_Y_train_dialog in zip(raw_X_train[train_indices], raw_Y_train[train_indices]):

            model_GoalName.hidden = model_GoalName.init_hidden()

            for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                goal_names = model_GoalName(indices, scores)

                for goal_name in goal_names:

                    goal_name = goal_name.unsqueeze(0)

                    loss_goal_name = GOAL_LOSS_FUNCTION(goal_name,
                                                        retrieve_gold_GoalName(raw_Y_train_turn,
                                                                               ontology = ontology,
                                                                               device = DEVICE))
                    dialogs_loss += loss_goal_name

        dialogs_loss.backward(retain_graph = False)

        optimizer_GoalNameModel.step()
    
    dev_tracker = make_tracker(model_GoalPricerange,
                   model_GoalArea,
                   model_GoalName,
                   model_GoalFood,
                   model_Requested,
                   model_Method,
                   raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
    dev_scores_dict = get_scores(dev_tracker, dataset = "dstc2_dev")

    logging.info("DEV Goal Name Acc: {}".format(dev_scores_dict["goal_name_accuracy"]))

    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_accuracy"],
                                                                 np.around(dev_scores_dict["goal_accuracy"], decimals = 2),
                                                                 dev_scores_dict["requested_accuracy"],
                                                                 np.around(dev_scores_dict["requested_accuracy"], decimals = 2),
                                                                 dev_scores_dict["method_accuracy"],
                                                                 np.around(dev_scores_dict["method_accuracy"], decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_l2"],
                                                                np.around(dev_scores_dict["goal_l2"], decimals = 2),
                                                                dev_scores_dict["requested_l2"],
                                                                np.around(dev_scores_dict["requested_l2"], decimals = 2),
                                                                dev_scores_dict["method_l2"],
                                                                np.around(dev_scores_dict["method_l2"], decimals = 2)))

    goal_name_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_name_accuracy"]))
    
    if goal_name_early_stopping.wait == 0:
        torch.save(model_GoalName.state_dict(), "model_PANFMR_incremental_GoalName.pt")
    
    if goal_name_early_stopping.stop_training:
        break

### Load Goal Name Model

In [ ]:
model_GoalName = GoalNameModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_name_dim = GOAL_NAME_DIM,
                               device = DEVICE)

model_GoalName = model_GoalName.to(DEVICE)
model_GoalName.load_state_dict(torch.load("model_PANFMR_incremental_GoalName.pt"))
model_GoalName.eval()

### Print scores

In [ ]:
dev_tracker = make_tracker(model_GoalPricerange,
                           model_GoalArea,
                           model_GoalName,
                           model_GoalFood,
                           model_Requested,
                           model_Method,
                           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
get_scores(dev_tracker, dataset = "dstc2_dev")

In [ ]:
test_tracker = make_tracker(model_GoalPricerange,
                            model_GoalArea,
                            model_GoalName,
                            model_GoalFood,
                            model_Requested,
                            model_Method,
                            raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = 1.0)
get_scores(test_tracker, dataset = "dstc2_test")

### Train Goal Food Model

In [ ]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

goal_food_early_stopping = EarlyStopping(patience = 6)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalFood = model_GoalFood.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        optimizer_GoalFoodModel.zero_grad()

        dialogs_loss = 0
    
        for raw_X_train_dialog, raw_Y_train_dialog in zip(raw_X_train[train_indices], raw_Y_train[train_indices]):

            model_GoalFood.hidden = model_GoalFood.init_hidden()

            for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                goal_foods = model_GoalFood(indices, scores)

                for goal_food in goal_foods:

                    goal_food = goal_food.unsqueeze(0)

                    loss_goal_food = GOAL_LOSS_FUNCTION(goal_food,
                                                        retrieve_gold_GoalFood(raw_Y_train_turn,
                                                                               ontology = ontology,
                                                                               device = DEVICE))
                    dialogs_loss += loss_goal_food

        dialogs_loss.backward(retain_graph = False)

        optimizer_GoalFoodModel.step()
    
    dev_tracker = make_tracker(model_GoalPricerange,
                   model_GoalArea,
                   model_GoalName,
                   model_GoalFood,
                   model_Requested,
                   model_Method,
                   raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
    dev_scores_dict = get_scores(dev_tracker, dataset = "dstc2_dev")

    logging.info("DEV Goal Food Acc: {}".format(dev_scores_dict["goal_food_accuracy"]))

    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_accuracy"],
                                                                 np.around(dev_scores_dict["goal_accuracy"], decimals = 2),
                                                                 dev_scores_dict["requested_accuracy"],
                                                                 np.around(dev_scores_dict["requested_accuracy"], decimals = 2),
                                                                 dev_scores_dict["method_accuracy"],
                                                                 np.around(dev_scores_dict["method_accuracy"], decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_l2"],
                                                                np.around(dev_scores_dict["goal_l2"], decimals = 2),
                                                                dev_scores_dict["requested_l2"],
                                                                np.around(dev_scores_dict["requested_l2"], decimals = 2),
                                                                dev_scores_dict["method_l2"],
                                                                np.around(dev_scores_dict["method_l2"], decimals = 2)))

    goal_food_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_food_accuracy"]))
    
    if goal_food_early_stopping.wait == 0:
        torch.save(model_GoalFood.state_dict(), "model_PANFMR_incremental_GoalFood.pt")
    
    if goal_food_early_stopping.stop_training:
        break

### Load Goal Food Model

In [ ]:
model_GoalFood = GoalFoodModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_food_dim = GOAL_FOOD_DIM,
                               device = DEVICE)

model_GoalFood = model_GoalFood.to(DEVICE)
model_GoalFood.load_state_dict(torch.load("model_PANFMR_incremental_GoalFood.pt"))
model_GoalFood.eval()

### Print scores

In [ ]:
dev_tracker = make_tracker(model_GoalPricerange,
                           model_GoalArea,
                           model_GoalName,
                           model_GoalFood,
                           model_Requested,
                           model_Method,
                           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
get_scores(dev_tracker, dataset = "dstc2_dev")

In [ ]:
test_tracker = make_tracker(model_GoalPricerange,
                            model_GoalArea,
                            model_GoalName,
                            model_GoalFood,
                            model_Requested,
                            model_Method,
                            raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = 1.0)
get_scores(test_tracker, dataset = "dstc2_test")

### Train Requested Model

In [ ]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

requested_early_stopping = EarlyStopping(patience = 6)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_Requested = model_Requested.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        optimizer_RequestedModel.zero_grad()

        dialogs_loss = 0
    
        for raw_X_train_dialog, raw_Y_train_dialog in zip(raw_X_train[train_indices], raw_Y_train[train_indices]):

            model_Requested.hidden = model_Requested.init_hidden()

            for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                requesteds = model_Requested(indices, scores)

                for requested in requesteds:

                    requested = requested.unsqueeze(0)

                    loss_requested = REQUESTED_LOSS_FUNCTION(requested,
                                                             retrieve_gold_Requested(raw_Y_train_turn,
                                                                                     ontology = ontology,
                                                                                     device = DEVICE))
                    dialogs_loss += loss_requested

        dialogs_loss.backward(retain_graph = False)

        optimizer_RequestedModel.step()
    
    dev_tracker = make_tracker(model_GoalPricerange,
                   model_GoalArea,
                   model_GoalName,
                   model_GoalFood,
                   model_Requested,
                   model_Method,
                   raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
    dev_scores_dict = get_scores(dev_tracker, dataset = "dstc2_dev")
    
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_accuracy"],
                                                                 np.around(dev_scores_dict["goal_accuracy"], decimals = 2),
                                                                 dev_scores_dict["requested_accuracy"],
                                                                 np.around(dev_scores_dict["requested_accuracy"], decimals = 2),
                                                                 dev_scores_dict["method_accuracy"],
                                                                 np.around(dev_scores_dict["method_accuracy"], decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_l2"],
                                                                np.around(dev_scores_dict["goal_l2"], decimals = 2),
                                                                dev_scores_dict["requested_l2"],
                                                                np.around(dev_scores_dict["requested_l2"], decimals = 2),
                                                                dev_scores_dict["method_l2"],
                                                                np.around(dev_scores_dict["method_l2"], decimals = 2)))

    requested_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["requested_accuracy"]))
    
    if requested_early_stopping.wait == 0:
        torch.save(model_Requested.state_dict(), "model_PANFMR_incremental_Requested.pt")
    
    if requested_early_stopping.stop_training:
        break

### Load Requested Model

In [ ]:
model_Requested = RequestedModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 requested_dim = REQUESTED_DIM,
                                 device = DEVICE)

model_Requested = model_Requested.to(DEVICE)
model_Requested.load_state_dict(torch.load("model_PANFMR_incremental_Requested.pt"))
model_Requested.eval()

### Print scores

In [ ]:
dev_tracker = make_tracker(model_GoalPricerange,
                           model_GoalArea,
                           model_GoalName,
                           model_GoalFood,
                           model_Requested,
                           model_Method,
                           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
get_scores(dev_tracker, dataset = "dstc2_dev")

In [ ]:
test_tracker = make_tracker(model_GoalPricerange,
                            model_GoalArea,
                            model_GoalName,
                            model_GoalFood,
                            model_Requested,
                            model_Method,
                            raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = 1.0)
get_scores(test_tracker, dataset = "dstc2_test")

### Train Method Model

In [ ]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

method_early_stopping = EarlyStopping(patience = 6)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_Method = model_Method.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
    
        optimizer_MethodModel.zero_grad()

        dialogs_loss = 0
    
        for raw_X_train_dialog, raw_Y_train_dialog in zip(raw_X_train[train_indices], raw_Y_train[train_indices]):

            model_Method.hidden = model_Method.init_hidden()

            for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                methods = model_Method(indices, scores)

                for method in methods:

                    method = method.unsqueeze(0)

                    loss_method = METHOD_LOSS_FUNCTION(method,
                                                       retrieve_gold_Method(raw_Y_train_turn,
                                                                            ontology = ontology,
                                                                            device = DEVICE))
                    dialogs_loss += loss_method

        dialogs_loss.backward(retain_graph = False)

        optimizer_MethodModel.step()
    
    dev_tracker = make_tracker(model_GoalPricerange,
                   model_GoalArea,
                   model_GoalName,
                   model_GoalFood,
                   model_Requested,
                   model_Method,
                   raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
    dev_scores_dict = get_scores(dev_tracker, dataset = "dstc2_dev")
    
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_accuracy"],
                                                                 np.around(dev_scores_dict["goal_accuracy"], decimals = 2),
                                                                 dev_scores_dict["requested_accuracy"],
                                                                 np.around(dev_scores_dict["requested_accuracy"], decimals = 2),
                                                                 dev_scores_dict["method_accuracy"],
                                                                 np.around(dev_scores_dict["method_accuracy"], decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_l2"],
                                                                np.around(dev_scores_dict["goal_l2"], decimals = 2),
                                                                dev_scores_dict["requested_l2"],
                                                                np.around(dev_scores_dict["requested_l2"], decimals = 2),
                                                                dev_scores_dict["method_l2"],
                                                                np.around(dev_scores_dict["method_l2"], decimals = 2)))

    method_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["method_accuracy"]))
    
    if method_early_stopping.wait == 0:
        torch.save(model_Method.state_dict(), "model_PANFMR_incremental_Method.pt")
    
    if method_early_stopping.stop_training:
        break

### Load Method Model

In [ ]:
model_Method = MethodModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           method_dim = METHOD_DIM,
                           device = DEVICE)

model_Method = model_Method.to(DEVICE)
model_Method.load_state_dict(torch.load("model_PANFMR_incremental_Method.pt"))
model_Method.eval()

### Print scores

In [ ]:
dev_tracker = make_tracker(model_GoalPricerange,
                           model_GoalArea,
                           model_GoalName,
                           model_GoalFood,
                           model_Requested,
                           model_Method,
                           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
get_scores(dev_tracker, dataset = "dstc2_dev")

In [ ]:
test_tracker = make_tracker(model_GoalPricerange,
                            model_GoalArea,
                            model_GoalName,
                            model_GoalFood,
                            model_Requested,
                            model_Method,
                            raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = 1.0)
get_scores(test_tracker, dataset = "dstc2_test")

### Plot incremental accuracies

In [ ]:
def plotly_plot_incremental(percentages, goal_accuracies, requested_accuracies, method_accuracies, dataset):
    plotly.offline.iplot({
                            "data": [Scatter(
                                            x = percentages,
                                            y = goal_accuracies,
                                            mode = "lines+markers",
                                            name = "{} Goal Accuracy".format(dataset),
                                            marker = dict(color = "#3498db")),
                                    Scatter(
                                            x = percentages,
                                            y = requested_accuracies,
                                            mode = "lines+markers",
                                            name = "{} Requested Accuracy".format(dataset),
                                            marker = dict(color = "#1abc9c")),
                                    Scatter(
                                            x = percentages,
                                            y = method_accuracies,
                                            mode = "lines+markers",
                                            name = "{} Method Accuracy".format(dataset),
                                            marker = dict(color = "#9b59b6"))],
                            "layout": Layout(
                                             title = "<b>Percentage - Accuracy</b>",
                                             xaxis = dict(title = "<b>Percentage</b>",
                                                          dtick = 0.1,
                                                          titlefont = dict(color = "#34495e")),
                                             yaxis = dict(title = "<b>Accuracy</b>",
                                                          titlefont = dict(color = "#34495e")),
                                             margin = Margin(b = 150))
                        })

In [ ]:
percentages = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

dev_goal_accuracies = []
dev_goal_l2s = []
dev_requested_accuracies = []
dev_requested_l2s = []
dev_method_accuracies = []
dev_method_l2s = []

test_goal_accuracies = []
test_goal_l2s = []
test_requested_accuracies = []
test_requested_l2s = []
test_method_accuracies = []
test_method_l2s = []

for percentage in tqdm_notebook(percentages, total = len(percentages)):
    
    dev_incremental_tracker = make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = percentage)
    
    dev_scores_dict = get_scores(dev_incremental_tracker, dataset = "dstc2_dev")
    
    dev_goal_accuracies.append(dev_scores_dict["goal_accuracy"])
    dev_goal_l2s.append(dev_scores_dict["goal_l2"])
    dev_requested_accuracies.append(dev_scores_dict["requested_accuracy"])
    dev_requested_l2s.append(dev_scores_dict["requested_l2"])
    dev_method_accuracies.append(dev_scores_dict["method_accuracy"])
    dev_method_l2s.append(dev_scores_dict["method_l2"])
    
    test_incremental_tracker = make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = percentage)
    
    test_scores_dict = get_scores(test_incremental_tracker, dataset = "dstc2_test")
    
    test_goal_accuracies.append(test_scores_dict["goal_accuracy"])
    test_goal_l2s.append(test_scores_dict["goal_l2"])
    test_requested_accuracies.append(test_scores_dict["requested_accuracy"])
    test_requested_l2s.append(test_scores_dict["requested_l2"])
    test_method_accuracies.append(test_scores_dict["method_accuracy"])
    test_method_l2s.append(test_scores_dict["method_l2"])

In [ ]:
plotly_plot_incremental(percentages, dev_goal_accuracies, dev_requested_accuracies, dev_method_accuracies, "Dev")

In [ ]:
plotly_plot_incremental(percentages, test_goal_accuracies, test_requested_accuracies, test_method_accuracies, "Test")

### SUPERVISED TURN TAKING

### Load Goal Pricerange Model

In [ ]:
model_GoalPricerange = GoalPricerangeModel(vocabulary_size = VOCABULARY_SIZE,
                                           embedding_dim = EMBEDDING_DIM,
                                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                           hidden_dim = HIDDEN_DIM,
                                           goal_pricerange_dim = GOAL_PRICERANGE_DIM,
                                           device = DEVICE)

model_GoalPricerange = model_GoalPricerange.to(DEVICE)
model_GoalPricerange.load_state_dict(torch.load("model_PANFMR_incremental_GoalPricerange.pt"))
model_GoalPricerange.eval()

### Load Goal Area Model

In [ ]:
model_GoalArea = GoalAreaModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_area_dim = GOAL_AREA_DIM,
                               device = DEVICE)

model_GoalArea = model_GoalArea.to(DEVICE)
model_GoalArea.load_state_dict(torch.load("model_PANFMR_incremental_GoalArea.pt"))
model_GoalArea.eval()

### Load Goal Name Model

In [ ]:
model_GoalName = GoalNameModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_name_dim = GOAL_NAME_DIM,
                               device = DEVICE)

model_GoalName = model_GoalName.to(DEVICE)
model_GoalName.load_state_dict(torch.load("model_PANFMR_incremental_GoalName.pt"))
model_GoalName.eval()

### Load Goal Food Model

In [ ]:
model_GoalFood = GoalFoodModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_food_dim = GOAL_FOOD_DIM,
                               device = DEVICE)

model_GoalFood = model_GoalFood.to(DEVICE)
model_GoalFood.load_state_dict(torch.load("model_PANFMR_incremental_GoalFood.pt"))
model_GoalFood.eval()

### Load Requested Model

In [ ]:
model_Requested = RequestedModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 requested_dim = REQUESTED_DIM,
                                 device = DEVICE)

model_Requested = model_Requested.to(DEVICE)
model_Requested.load_state_dict(torch.load("model_PANFMR_incremental_Requested.pt"))
model_Requested.eval()

### Load Method Model

In [ ]:
model_Method = MethodModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           method_dim = METHOD_DIM,
                           device = DEVICE)

model_Method = model_Method.to(DEVICE)
model_Method.load_state_dict(torch.load("model_PANFMR_incremental_Method.pt"))
model_Method.eval()

### Print scores

In [ ]:
dev_tracker = make_tracker(model_GoalPricerange,
                           model_GoalArea,
                           model_GoalName,
                           model_GoalFood,
                           model_Requested,
                           model_Method,
                           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
get_scores(dev_tracker, dataset = "dstc2_dev")

In [ ]:
test_tracker = make_tracker(model_GoalPricerange,
                            model_GoalArea,
                            model_GoalName,
                            model_GoalFood,
                            model_Requested,
                            model_Method,
                            raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = 1.0)
get_scores(test_tracker, dataset = "dstc2_test")

### Predictor utilities

In [ ]:
def compute_predictor_loss(model, raw_X, predictor_Y, objective, device):
    
    model = model.eval()
    
    loss = 0
                           
    for raw_X_dialog, predictor_Y_dialog in zip(raw_X, predictor_Y):

            model.hidden = model.init_hidden()

            for raw_X_turn, predictor_Y_turn in zip(raw_X_dialog["turns"], predictor_Y_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)

                model_predictions = model(indices, scores)

                for model_prediction, gold_prediction in zip(model_predictions, predictor_Y_turn[objective]):

                    model_prediction = model_prediction.unsqueeze(0)
                    
                    gold_prediction = torch.tensor([gold_prediction], dtype = torch.long, device = DEVICE)

                    prediction_loss = PREDICTOR_LOSS_FUNCTION(model_prediction, gold_prediction)

                    loss += prediction_loss
    return loss

def get_gaps_vector(vector):
    gaps_vector = []
    
    for value in vector:
        if value == vector[-1]:
            gaps_vector.append(0)
        else:
            gaps_vector.append(1)
    
    return gaps_vector


def get_predictor_score(dataset, session_id, turns_so_far):
    start_time = time.time()
    
    tracker_json = {}
    tracker_json["dataset"] = dataset
    tracker_json["sessions"] = []
    
    session = {}
    session["session-id"] = session_id
    session["turns"] = turns_so_far
    
    tracker_json["sessions"].append(session)
        
    end_time = time.time()
    
    tracker_json["wall-time"] = end_time - start_time
    
    return get_scores(tracker_json, dataset, predictor = True)
    

def get_predictor_Y(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, dataset, ontology):
    
    predictor_Y = []
    
    if dataset == "dstc2_train":
        raw_X = raw_X_train
        raw_Y = raw_Y_train
    elif dataset == "dstc2_dev":
        raw_X = raw_X_dev
        raw_Y = raw_Y_dev
    else: # dataset == "dstc2_test"
        raw_X = raw_X_test
        raw_Y = raw_Y_test
    
    model_GoalPricerange = model_GoalPricerange.eval()
    model_GoalArea = model_GoalArea.eval()
    model_GoalName = model_GoalName.eval()
    model_GoalFood = model_GoalFood.eval()
    model_Requested = model_Requested.eval()
    model_Method = model_Method.eval()
    
    for raw_X_dialog, raw_Y_dialog in tqdm_notebook(zip(raw_X, raw_Y), total = len(raw_X)):
        
        assert(raw_X_dialog["session-id"] == raw_Y_dialog["session-id"])
        
        predictor_Y_dialog = {}
        predictor_Y_dialog["session-id"] = raw_Y_dialog["session-id"]
        predictor_Y_dialog["turns"] = []
        
        model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
        model_GoalArea.hidden = model_GoalArea.init_hidden()
        model_GoalName.hidden = model_GoalName.init_hidden()
        model_GoalFood.hidden = model_GoalFood.init_hidden()
        model_Requested.hidden = model_Requested.init_hidden()
        model_Method.hidden = model_Method.init_hidden()

        turns_so_far = []

        for raw_X_turn, raw_Y_turn in zip(raw_X_dialog["turns"], raw_Y_dialog["turns"]):
            
            predictor_Y_turn = {}
            predictor_Y_turn["goal_pricerange_predictor"] = []
            predictor_Y_turn["goal_area_predictor"] = []
            predictor_Y_turn["goal_name_predictor"] = []
            predictor_Y_turn["goal_food_predictor"] = []
            predictor_Y_turn["requested_predictor"] = []
            predictor_Y_turn["method_predictor"] = []
            
            turn = {}
            turn["goal-labels"] = {}

            indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)

            goal_foods = model_GoalFood(indices, scores)
            goal_priceranges = model_GoalPricerange(indices, scores)
            goal_names = model_GoalName(indices, scores)
            goal_areas = model_GoalArea(indices, scores)
            requesteds = model_Requested(indices, scores)
            methods = model_Method(indices, scores)
            
            for goal_food, goal_pricerange, goal_name, goal_area, requested, method in zip(goal_foods, goal_priceranges, goal_names, goal_areas, requesteds, methods):
                
                turns_so_far_copy = copy.deepcopy(turns_so_far)
                
                turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food, ontology)
                turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange, ontology)
                turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name, ontology)
                turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area, ontology)
                turn["requested-slots"] = retrieve_output_Requested(requested, ontology)
                turn["method-label"] = retrieve_output_Method(method, ontology)
            
                turns_so_far_copy.append(turn)
                
                scores_dict = get_predictor_score(dataset, raw_X_dialog["session-id"], turns_so_far_copy)
                
                predictor_Y_turn["goal_pricerange_predictor"].append(scores_dict["goal_pricerange_accuracy"])
                predictor_Y_turn["goal_area_predictor"].append(scores_dict["goal_area_accuracy"])
                predictor_Y_turn["goal_name_predictor"].append(scores_dict["goal_name_accuracy"])
                predictor_Y_turn["goal_food_predictor"].append(scores_dict["goal_food_accuracy"])
                predictor_Y_turn["requested_predictor"].append(scores_dict["requested_accuracy"])
                predictor_Y_turn["method_predictor"].append(scores_dict["method_accuracy"])
                                
            turns_so_far.append(turn)
                
            predictor_Y_turn["goal_pricerange_predictor"] = get_gaps_vector(predictor_Y_turn["goal_pricerange_predictor"])
            predictor_Y_turn["goal_area_predictor"] = get_gaps_vector(predictor_Y_turn["goal_area_predictor"])
            predictor_Y_turn["goal_name_predictor"] = get_gaps_vector(predictor_Y_turn["goal_name_predictor"])
            predictor_Y_turn["goal_food_predictor"] = get_gaps_vector(predictor_Y_turn["goal_food_predictor"])
            predictor_Y_turn["requested_predictor"] = get_gaps_vector(predictor_Y_turn["requested_predictor"])
            predictor_Y_turn["method_predictor"] = get_gaps_vector(predictor_Y_turn["method_predictor"])
            
            predictor_Y_dialog["turns"].append(predictor_Y_turn)
            
        predictor_Y.append(predictor_Y_dialog) 
    
    return np.array(predictor_Y)

### Compute development set predictor labels

In [ ]:
dev_predictor_Y = get_predictor_Y(model_GoalPricerange,
                                  model_GoalArea,
                                  model_GoalName,
                                  model_GoalFood,
                                  model_Requested,
                                  model_Method,
                                  "dstc2_dev",
                                  ontology)

### Save development set predictor labels

In [ ]:
np.save("dev_predictor_Y", dev_predictor_Y)

### Load development set predictor labels

In [ ]:
dev_predictor_Y = np.load("dev_predictor_Y.npy")

### Compute train set predictor labels

In [ ]:
train_predictor_Y = get_predictor_Y(model_GoalPricerange,
                                    model_GoalArea,
                                    model_GoalName,
                                    model_GoalFood,
                                    model_Requested,
                                    model_Method,
                                    "dstc2_train",
                                    ontology)

### Save train set predictor labels

In [ ]:
np.save("train_predictor_Y", train_predictor_Y)

### Load train set predictor labels

In [ ]:
train_predictor_Y = np.load("train_predictor_Y.npy")

### Goal Pricerange Predictor Model

In [ ]:
class GoalPricerangePredictorModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_pricerange_predictor_dim,
                 device):
        
        super(GoalPricerangePredictorModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_pricerange_predictor_dim = goal_pricerange_predictor_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_pricerange_predictor_classifier = nn.Linear(in_features = hidden_dim,
                                                              out_features = goal_pricerange_predictor_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_pricerange_predictor = F.log_softmax(self.goal_pricerange_predictor_classifier(lstm_out).view(-1, self.goal_pricerange_predictor_dim), dim = 1) 
        
        return goal_pricerange_predictor

model_GoalPricerangePredictor = GoalPricerangePredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                             embedding_dim = EMBEDDING_DIM,
                                                             altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                             hidden_dim = HIDDEN_DIM,
                                                             goal_pricerange_predictor_dim = PREDICTOR_DIM,
                                                             device = DEVICE)

model_GoalPricerangePredictor = model_GoalPricerangePredictor.to(DEVICE)

optimizer_GoalPricerangePredictorModel = optim.Adam(model_GoalPricerangePredictor.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Area Predictor Model

In [ ]:
class GoalAreaPredictorModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_area_predictor_dim,
                 device):
        
        super(GoalAreaPredictorModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_area_predictor_dim = goal_area_predictor_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_area_predictor_classifier = nn.Linear(in_features = hidden_dim,
                                                        out_features = goal_area_predictor_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_area_predictor = F.log_softmax(self.goal_area_predictor_classifier(lstm_out).view(-1, self.goal_area_predictor_dim), dim = 1)
        
        return goal_area_predictor

model_GoalAreaPredictor = GoalAreaPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                 embedding_dim = EMBEDDING_DIM,
                                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                 hidden_dim = HIDDEN_DIM,
                                                 goal_area_predictor_dim = PREDICTOR_DIM,
                                                 device = DEVICE)

model_GoalAreaPredictor = model_GoalAreaPredictor.to(DEVICE)

optimizer_GoalAreaPredictorModel = optim.Adam(model_GoalAreaPredictor.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Name Predictor Model

In [ ]:
class GoalNamePredictorModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_name_predictor_dim,
                 device):
        
        super(GoalNamePredictorModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_name_predictor_dim = goal_name_predictor_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_name_predictor_classifier = nn.Linear(in_features = hidden_dim,
                                                        out_features = goal_name_predictor_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_name_predictor = F.log_softmax(self.goal_name_predictor_classifier(lstm_out).view(-1, self.goal_name_predictor_dim), dim = 1)
        
        return goal_name_predictor

model_GoalNamePredictor = GoalNamePredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                 embedding_dim = EMBEDDING_DIM,
                                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                 hidden_dim = HIDDEN_DIM,
                                                 goal_name_predictor_dim = PREDICTOR_DIM,
                                                 device = DEVICE)

model_GoalNamePredictor = model_GoalNamePredictor.to(DEVICE)

optimizer_GoalNamePredictorModel = optim.Adam(model_GoalNamePredictor.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Food Predictor Model

In [ ]:
class GoalFoodPredictorModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_food_predictor_dim,
                 device):
        
        super(GoalFoodPredictorModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_food_predictor_dim = goal_food_predictor_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_food_predictor_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_food_predictor_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_food_predictor = F.log_softmax(self.goal_food_predictor_classifier(lstm_out).view(-1, self.goal_food_predictor_dim), dim = 1)        
        
        return goal_food_predictor

model_GoalFoodPredictor = GoalFoodPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                 embedding_dim = EMBEDDING_DIM,
                                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                 hidden_dim = HIDDEN_DIM,
                                                 goal_food_predictor_dim = PREDICTOR_DIM,
                                                 device = DEVICE)

model_GoalFoodPredictor = model_GoalFoodPredictor.to(DEVICE)

optimizer_GoalFoodPredictorModel = optim.Adam(model_GoalFoodPredictor.parameters(), lr = 1e-3, amsgrad = True) 

### Requested Predictor Model

In [ ]:
class RequestedPredictorModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 requested_predictor_dim,
                 device):
        
        super(RequestedPredictorModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.requested_predictor_dim = requested_predictor_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.requested_predictor_classifier = nn.Linear(in_features = hidden_dim,
                                                        out_features = requested_predictor_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        requested_predictor = F.log_softmax(self.requested_predictor_classifier(lstm_out).view(-1, self.requested_predictor_dim), dim = 1)
        
        return requested_predictor

model_RequestedPredictor = RequestedPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                   embedding_dim = EMBEDDING_DIM,
                                                   altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                   hidden_dim = HIDDEN_DIM,
                                                   requested_predictor_dim = PREDICTOR_DIM,
                                                   device = DEVICE)

model_RequestedPredictor = model_RequestedPredictor.to(DEVICE)

optimizer_RequestedPredictorModel = optim.Adam(model_RequestedPredictor.parameters(), lr = 1e-3, amsgrad = True)

### Method Predictor Model

In [ ]:
class MethodPredictorModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 method_predictor_dim,
                 device):
        
        super(MethodPredictorModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.method_predictor_dim = method_predictor_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.method_predictor_classifier = nn.Linear(in_features = hidden_dim,
                                                     out_features = method_predictor_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        method_predictor = F.log_softmax(self.method_predictor_classifier(lstm_out).view(-1, self.method_predictor_dim), dim = 1)
        
        return method_predictor

model_MethodPredictor = MethodPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                             embedding_dim = EMBEDDING_DIM,
                                             altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                             hidden_dim = HIDDEN_DIM,
                                             method_predictor_dim = PREDICTOR_DIM,
                                             device = DEVICE)

model_MethodPredictor = model_MethodPredictor.to(DEVICE)

optimizer_MethodPredictorModel = optim.Adam(model_MethodPredictor.parameters(), lr = 1e-3, amsgrad = True) 

### Train Goal Pricerange Predictor Model

In [ ]:
goal_pricerange_predictor_early_stopping = EarlyStopping(patience = 6)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalPricerangePredictor = model_GoalPricerangePredictor.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        optimizer_GoalPricerangePredictorModel.zero_grad()
            
        dialogs_loss = 0
    
        for raw_X_train_dialog, train_predictor_Y_dialog in zip(raw_X_train[train_indices], train_predictor_Y[train_indices]):

            model_GoalPricerangePredictor.hidden = model_GoalPricerangePredictor.init_hidden()

            for raw_X_train_turn, train_predictor_Y_turn in zip(raw_X_train_dialog["turns"], train_predictor_Y_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                goal_priceranges_predictor = model_GoalPricerangePredictor(indices, scores)

                for goal_pricerange_predictor, gold_goal_pricerange_predictor in zip(goal_priceranges_predictor, train_predictor_Y_turn["goal_pricerange_predictor"]):

                    goal_pricerange_predictor = goal_pricerange_predictor.unsqueeze(0)
                    
                    gold_goal_pricerange_predictor = torch.tensor([gold_goal_pricerange_predictor], dtype = torch.long, device = DEVICE)
                    
                    loss_goal_pricerange_predictor = PREDICTOR_LOSS_FUNCTION(goal_pricerange_predictor, gold_goal_pricerange_predictor)

                    dialogs_loss += loss_goal_pricerange_predictor
                
        dialogs_loss.backward(retain_graph = False)

        optimizer_GoalPricerangePredictorModel.step()
    
    train_loss = compute_predictor_loss(model_GoalPricerangePredictor, raw_X_train, train_predictor_Y, objective = "goal_pricerange_predictor", device = DEVICE)
    dev_loss = compute_predictor_loss(model_GoalPricerangePredictor, raw_X_dev, dev_predictor_Y, objective = "goal_pricerange_predictor", device = DEVICE)
    
    logging.info("TRAIN LOSS: {}\tDEV LOSS: {}".format(train_loss.item(), dev_loss.item()))

    goal_pricerange_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = -dev_loss.item())
    
    if goal_pricerange_predictor_early_stopping.wait == 0:
        torch.save(model_GoalPricerangePredictor.state_dict(), "model_PANFMR_incremental_GoalPricerangePredictor.pt")
    
    if goal_pricerange_predictor_early_stopping.stop_training:
        break

### Load Goal Pricerange Predictor Model

In [ ]:
model_GoalPricerangePredictor = GoalPricerangePredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                             embedding_dim = EMBEDDING_DIM,
                                                             altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                             hidden_dim = HIDDEN_DIM,
                                                             goal_pricerange_predictor_dim = PREDICTOR_DIM,
                                                             device = DEVICE)

model_GoalPricerangePredictor = model_GoalPricerangePredictor.to(DEVICE)
model_GoalPricerangePredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalPricerangePredictor.pt"))
model_GoalPricerangePredictor.eval()

### Train Goal Area Predictor Model

In [ ]:
goal_area_predictor_early_stopping = EarlyStopping(patience = 6)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalAreaPredictor = model_GoalAreaPredictor.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        optimizer_GoalAreaPredictorModel.zero_grad()
            
        dialogs_loss = 0
    
        for raw_X_train_dialog, train_predictor_Y_dialog in zip(raw_X_train[train_indices], train_predictor_Y[train_indices]):

            model_GoalAreaPredictor.hidden = model_GoalAreaPredictor.init_hidden()

            for raw_X_train_turn, train_predictor_Y_turn in zip(raw_X_train_dialog["turns"], train_predictor_Y_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                goal_areas_predictor = model_GoalAreaPredictor(indices, scores)

                for goal_area_predictor, gold_goal_area_predictor in zip(goal_areas_predictor, train_predictor_Y_turn["goal_area_predictor"]):

                    goal_area_predictor = goal_area_predictor.unsqueeze(0)
                    
                    gold_goal_area_predictor = torch.tensor([gold_goal_area_predictor], dtype = torch.long, device = DEVICE)
                    
                    loss_goal_area_predictor = PREDICTOR_LOSS_FUNCTION(goal_area_predictor, gold_goal_area_predictor)

                    dialogs_loss += loss_goal_area_predictor
                
        dialogs_loss.backward(retain_graph = False)

        optimizer_GoalAreaPredictorModel.step()
    
    train_loss = compute_predictor_loss(model_GoalAreaPredictor, raw_X_train, train_predictor_Y, objective = "goal_area_predictor", device = DEVICE)
    dev_loss = compute_predictor_loss(model_GoalAreaPredictor, raw_X_dev, dev_predictor_Y, objective = "goal_area_predictor", device = DEVICE)
    
    logging.info("TRAIN LOSS: {}\tDEV LOSS: {}".format(train_loss.item(), dev_loss.item()))

    goal_area_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = -dev_loss.item())
    
    if goal_area_predictor_early_stopping.wait == 0:
        torch.save(model_GoalAreaPredictor.state_dict(), "model_PANFMR_incremental_GoalAreaPredictor.pt")
    
    if goal_area_predictor_early_stopping.stop_training:
        break

### Load Goal Area Predictor Model

In [ ]:
model_GoalAreaPredictor = GoalAreaPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                 embedding_dim = EMBEDDING_DIM,
                                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                 hidden_dim = HIDDEN_DIM,
                                                 goal_area_predictor_dim = PREDICTOR_DIM,
                                                 device = DEVICE)

model_GoalAreaPredictor = model_GoalAreaPredictor.to(DEVICE)
model_GoalAreaPredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalAreaPredictor.pt"))
model_GoalAreaPredictor.eval()

### Train Goal Name Predictor Model

In [ ]:
goal_name_predictor_early_stopping = EarlyStopping(patience = 6)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalNamePredictor = model_GoalNamePredictor.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        optimizer_GoalNamePredictorModel.zero_grad()
            
        dialogs_loss = 0
    
        for raw_X_train_dialog, train_predictor_Y_dialog in zip(raw_X_train[train_indices], train_predictor_Y[train_indices]):

            model_GoalNamePredictor.hidden = model_GoalNamePredictor.init_hidden()

            for raw_X_train_turn, train_predictor_Y_turn in zip(raw_X_train_dialog["turns"], train_predictor_Y_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                goal_names_predictor = model_GoalNamePredictor(indices, scores)

                for goal_name_predictor, gold_goal_name_predictor in zip(goal_names_predictor, train_predictor_Y_turn["goal_name_predictor"]):

                    goal_name_predictor = goal_name_predictor.unsqueeze(0)
                    
                    gold_goal_name_predictor = torch.tensor([gold_goal_name_predictor], dtype = torch.long, device = DEVICE)
                    
                    loss_goal_name_predictor = PREDICTOR_LOSS_FUNCTION(goal_name_predictor, gold_goal_name_predictor)

                    dialogs_loss += loss_goal_name_predictor
                
        dialogs_loss.backward(retain_graph = False)

        optimizer_GoalNamePredictorModel.step()
    
    train_loss = compute_predictor_loss(model_GoalNamePredictor, raw_X_train, train_predictor_Y, objective = "goal_name_predictor", device = DEVICE)
    dev_loss = compute_predictor_loss(model_GoalNamePredictor, raw_X_dev, dev_predictor_Y, objective = "goal_name_predictor", device = DEVICE)
    
    logging.info("TRAIN LOSS: {}\tDEV LOSS: {}".format(train_loss.item(), dev_loss.item()))

    goal_name_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = -dev_loss.item())
    
    if goal_name_predictor_early_stopping.wait == 0:
        torch.save(model_GoalNamePredictor.state_dict(), "model_PANFMR_incremental_GoalNamePredictor.pt")
    
    if goal_name_predictor_early_stopping.stop_training:
        break

### Load Goal Name Predictor Model

In [ ]:
model_GoalNamePredictor = GoalNamePredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                 embedding_dim = EMBEDDING_DIM,
                                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                 hidden_dim = HIDDEN_DIM,
                                                 goal_name_predictor_dim = PREDICTOR_DIM,
                                                 device = DEVICE)

model_GoalNamePredictor = model_GoalNamePredictor.to(DEVICE)
model_GoalNamePredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalNamePredictor.pt"))
model_GoalNamePredictor.eval()

### Train Goal Food Predictor Model

In [ ]:
goal_food_predictor_early_stopping = EarlyStopping(patience = 6)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalFoodPredictor = model_GoalFoodPredictor.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        optimizer_GoalFoodPredictorModel.zero_grad()
            
        dialogs_loss = 0
    
        for raw_X_train_dialog, train_predictor_Y_dialog in zip(raw_X_train[train_indices], train_predictor_Y[train_indices]):

            model_GoalFoodPredictor.hidden = model_GoalFoodPredictor.init_hidden()

            for raw_X_train_turn, train_predictor_Y_turn in zip(raw_X_train_dialog["turns"], train_predictor_Y_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                goal_foods_predictor = model_GoalFoodPredictor(indices, scores)

                for goal_food_predictor, gold_goal_food_predictor in zip(goal_foods_predictor, train_predictor_Y_turn["goal_food_predictor"]):

                    goal_food_predictor = goal_food_predictor.unsqueeze(0)
                    
                    gold_goal_food_predictor = torch.tensor([gold_goal_food_predictor], dtype = torch.long, device = DEVICE)
                    
                    loss_goal_food_predictor = PREDICTOR_LOSS_FUNCTION(goal_food_predictor, gold_goal_food_predictor)

                    dialogs_loss += loss_goal_food_predictor
                
        dialogs_loss.backward(retain_graph = False)

        optimizer_GoalFoodPredictorModel.step()
    
    train_loss = compute_predictor_loss(model_GoalFoodPredictor, raw_X_train, train_predictor_Y, objective = "goal_food_predictor", device = DEVICE)
    dev_loss = compute_predictor_loss(model_GoalFoodPredictor, raw_X_dev, dev_predictor_Y, objective = "goal_food_predictor", device = DEVICE)
    
    logging.info("TRAIN LOSS: {}\tDEV LOSS: {}".format(train_loss.item(), dev_loss.item()))

    goal_food_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = -dev_loss.item())
    
    if goal_food_predictor_early_stopping.wait == 0:
        torch.save(model_GoalFoodPredictor.state_dict(), "model_PANFMR_incremental_GoalFoodPredictor.pt")
    
    if goal_food_predictor_early_stopping.stop_training:
        break

### Load Goal Food Predictor

In [ ]:
model_GoalFoodPredictor = GoalFoodPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                 embedding_dim = EMBEDDING_DIM,
                                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                 hidden_dim = HIDDEN_DIM,
                                                 goal_food_predictor_dim = PREDICTOR_DIM,
                                                 device = DEVICE)

model_GoalFoodPredictor = model_GoalFoodPredictor.to(DEVICE)
model_GoalFoodPredictor.load_state_dict(torch.load("model_PANFMR_incremental_GoalFoodPredictor.pt"))
model_GoalFoodPredictor.eval()

### Train Requested Predictor Model

In [ ]:
requested_predictor_early_stopping = EarlyStopping(patience = 6)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_RequestedPredictor = model_RequestedPredictor.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        optimizer_RequestedPredictorModel.zero_grad()
            
        dialogs_loss = 0
    
        for raw_X_train_dialog, train_predictor_Y_dialog in zip(raw_X_train[train_indices], train_predictor_Y[train_indices]):

            model_RequestedPredictor.hidden = model_RequestedPredictor.init_hidden()

            for raw_X_train_turn, train_predictor_Y_turn in zip(raw_X_train_dialog["turns"], train_predictor_Y_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                requesteds_predictor = model_RequestedPredictor(indices, scores)

                for requested_predictor, gold_requested_predictor in zip(requesteds_predictor, train_predictor_Y_turn["requested_predictor"]):

                    requested_predictor = requested_predictor.unsqueeze(0)
                    
                    gold_requested_predictor = torch.tensor([gold_requested_predictor], dtype = torch.long, device = DEVICE)
                    
                    loss_requested_predictor = PREDICTOR_LOSS_FUNCTION(requested_predictor, gold_requested_predictor)

                    dialogs_loss += loss_requested_predictor
                
        dialogs_loss.backward(retain_graph = False)

        optimizer_RequestedPredictorModel.step()
    
    train_loss = compute_predictor_loss(model_RequestedPredictor, raw_X_train, train_predictor_Y, objective = "requested_predictor", device = DEVICE)
    dev_loss = compute_predictor_loss(model_RequestedPredictor, raw_X_dev, dev_predictor_Y, objective = "requested_predictor", device = DEVICE)
    
    logging.info("TRAIN LOSS: {}\tDEV LOSS: {}".format(train_loss.item(), dev_loss.item()))

    requested_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = -dev_loss.item())
    
    if requested_predictor_early_stopping.wait == 0:
        torch.save(model_RequestedPredictor.state_dict(), "model_PANFMR_incremental_RequestedPredictor.pt")
    
    if requested_predictor_early_stopping.stop_training:
        break

### Load Requested Predictor Model

In [ ]:
model_RequestedPredictor = RequestedPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                                   embedding_dim = EMBEDDING_DIM,
                                                   altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                                   hidden_dim = HIDDEN_DIM,
                                                   requested_predictor_dim = PREDICTOR_DIM,
                                                   device = DEVICE)

model_RequestedPredictor = model_RequestedPredictor.to(DEVICE)
model_RequestedPredictor.load_state_dict(torch.load("model_PANFMR_incremental_RequestedPredictor.pt"))
model_RequestedPredictor.eval()

### Train Method Predictor Model

In [ ]:
method_predictor_early_stopping = EarlyStopping(patience = 6)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_MethodPredictor = model_MethodPredictor.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        optimizer_MethodPredictorModel.zero_grad()
            
        dialogs_loss = 0
    
        for raw_X_train_dialog, train_predictor_Y_dialog in zip(raw_X_train[train_indices], train_predictor_Y[train_indices]):

            model_MethodPredictor.hidden = model_MethodPredictor.init_hidden()

            for raw_X_train_turn, train_predictor_Y_turn in zip(raw_X_train_dialog["turns"], train_predictor_Y_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

                methods_predictor = model_MethodPredictor(indices, scores)

                for method_predictor, gold_method_predictor in zip(methods_predictor, train_predictor_Y_turn["method_predictor"]):

                    method_predictor = method_predictor.unsqueeze(0)
                    
                    gold_method_predictor = torch.tensor([gold_method_predictor], dtype = torch.long, device = DEVICE)
                    
                    loss_method_predictor = PREDICTOR_LOSS_FUNCTION(method_predictor, gold_method_predictor)

                    dialogs_loss += loss_method_predictor
                
        dialogs_loss.backward(retain_graph = False)

        optimizer_MethodPredictorModel.step()
    
    train_loss = compute_predictor_loss(model_MethodPredictor, raw_X_train, train_predictor_Y, objective = "method_predictor", device = DEVICE)
    dev_loss = compute_predictor_loss(model_MethodPredictor, raw_X_dev, dev_predictor_Y, objective = "method_predictor", device = DEVICE)
    
    logging.info("TRAIN LOSS: {}\tDEV LOSS: {}".format(train_loss.item(), dev_loss.item()))

    method_predictor_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = -dev_loss.item())
    
    if method_predictor_early_stopping.wait == 0:
        torch.save(model_MethodPredictor.state_dict(), "model_PANFMR_incremental_MethodPredictor.pt")
    
    if method_predictor_early_stopping.stop_training:
        break

### Load Method Predictor

In [ ]:
model_MethodPredictor = MethodPredictorModel(vocabulary_size = VOCABULARY_SIZE,
                                             embedding_dim = EMBEDDING_DIM,
                                             altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                             hidden_dim = HIDDEN_DIM,
                                             method_predictor_dim = PREDICTOR_DIM,
                                             device = DEVICE)

model_MethodPredictor = model_MethodPredictor.to(DEVICE)
model_MethodPredictor.load_state_dict(torch.load("model_PANFMR_incremental_MethodPredictor.pt"))
model_MethodPredictor.eval()

### Print scores

In [ ]:
def make_predictor_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method,
                           model_GoalPricerangePredictor, model_GoalAreaPredictor, model_GoalNamePredictor, model_GoalFoodPredictor, model_RequestedPredictor, model_MethodPredictor,
                           raw_X, raw_Y, dataset):
    
    model_GoalPricerange = model_GoalPricerange.eval()
    model_GoalArea = model_GoalArea.eval()
    model_GoalName = model_GoalName.eval()
    model_GoalFood = model_GoalFood.eval()
    model_Requested = model_Requested.eval()
    model_Method = model_Method.eval()
    
    model_GoalPricerangePredictor = model_GoalPricerangePredictor.eval()
    model_GoalAreaPredictor = model_GoalAreaPredictor.eval()
    model_GoalNamePredictor = model_GoalNamePredictor.eval()
    model_GoalFoodPredictor = model_GoalFoodPredictor.eval()
    model_RequestedPredictor = model_RequestedPredictor.eval()
    model_MethodPredictor = model_MethodPredictor.eval()
    
    with torch.no_grad():
        
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        for raw_X_dialog, raw_Y_dialog in tqdm_notebook(zip(raw_X, raw_Y), total = len(raw_X)):
            
            model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
            model_GoalArea.hidden = model_GoalArea.init_hidden()
            model_GoalName.hidden = model_GoalName.init_hidden()
            model_GoalFood.hidden = model_GoalFood.init_hidden()
            model_Requested.hidden = model_Requested.init_hidden()
            model_Method.hidden = model_Method.init_hidden()
            
            model_GoalPricerangePredictor.hidden = model_GoalPricerangePredictor.init_hidden()
            model_GoalAreaPredictor.hidden = model_GoalAreaPredictor.init_hidden()
            model_GoalNamePredictor.hidden = model_GoalNamePredictor.init_hidden()
            model_GoalFoodPredictor.hidden = model_GoalFoodPredictor.init_hidden()
            model_RequestedPredictor.hidden = model_RequestedPredictor.init_hidden()
            model_MethodPredictor.hidden = model_MethodPredictor.init_hidden()
            
            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for raw_X_turn, raw_Y_turn in zip(raw_X_dialog["turns"], raw_Y_dialog["turns"]):
                
                turn = {}
                turn["goal-labels"] = {}

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)
                                
                goal_priceranges = model_GoalPricerange(indices, scores)
                goal_areas = model_GoalArea(indices, scores)
                goal_names = model_GoalName(indices, scores)
                goal_foods = model_GoalFood(indices, scores)
                requesteds = model_Requested(indices, scores)
                methods = model_Method(indices, scores)
                
                goal_priceranges_predictor = model_GoalPricerangePredictor(indices, scores)
                goal_areas_predictor = model_GoalAreaPredictor(indices, scores)
                goal_names_predictor = model_GoalNamePredictor(indices, scores)
                goal_foods_predictor = model_GoalFoodPredictor(indices, scores)
                requesteds_predictor = model_RequestedPredictor(indices, scores)
                methods_predictor = model_MethodPredictor(indices, scores)
                
                for goal_pricerange, goal_area, goal_name, goal_food, requested, method, \
                    goal_pricerange_predictor, goal_area_predictor, goal_name_predictor, goal_food_predictor, requested_predictor, method_predictor in \
                    zip(goal_priceranges, goal_areas, goal_names, goal_foods, requesteds, methods, \
                       goal_priceranges_predictor, goal_areas_predictor, goal_names_predictor, goal_foods_predictor, requesteds_predictor, methods_predictor):
                    
                    goal_pricerange_predictor = torch.exp(goal_pricerange_predictor)[0].item()
                    goal_area_predictor = torch.exp(goal_area_predictor)[0].item()
                    goal_name_predictor = torch.exp(goal_name_predictor)[0].item()
                    goal_food_predictor = torch.exp(goal_food_predictor)[0].item()
                    requested_predictor = torch.exp(requested_predictor)[0].item()
                    method_predictor = torch.exp(method_predictor)[0].item()
                    
                    if (goal_pricerange_predictor >= 0.5) and (goal_area_predictor >= 0.5) and (goal_name_predictor >= 0.5) and \
                        (goal_food_predictor >= 0.5) and (requested_predictor >= 0.5) and (method_predictor >= 0.5):
                        turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange, ontology)
                        turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name, ontology)
                        turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area, ontology)
                        turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food, ontology)
                        turn["requested-slots"] = retrieve_output_Requested(requested, ontology)
                        turn["method-label"] = retrieve_output_Method(method, ontology)
                
                        session["turns"].append(turn)
                        break
                
            tracker_json["sessions"].append(session)
            
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        
        return tracker_json

In [ ]:
dev_tracker = make_predictor_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method,
                                     model_GoalPricerangePredictor, model_GoalAreaPredictor, model_GoalNamePredictor, model_GoalFoodPredictor, model_RequestedPredictor, model_MethodPredictor,
                                     raw_X_dev, raw_Y_dev, "dstc2_dev")
get_scores(dev_tracker, dataset = "dstc2_dev")